# 허깅페이스 BERT 영화 리뷰 감정 분류 Tutorial

reference : https://huggingface.co/docs/transformers/tasks/sequence_classification

## 데이터셋 로드

In [1]:
import transformers
from datasets import load_dataset

# dataset load
imdb = load_dataset("imdb") 

데이터 구성 확인

In [2]:
print(imdb['train'])
print(imdb['train'][0])

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered 

## 토크나이저 로드 및 토큰화 수행

In [3]:
from transformers import AutoTokenizer

# BERT 모델 구조에 맞는 토크나이저 로드. bert-base-uncased는 BERT 모델의 기본 버전
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 토큰화 수행
tokenized_imdb = imdb.map(lambda x : tokenizer(x['text'], truncation=True), batched=True)
"""
imbd는 load_dataset("imdb")를 통해 로드된 Dataset 객체이다.
Dataset.map(f)는 입력 데이터셋의 각 원소에 함수 f을 적용하여 새로운 데이터셋을 생성해주는 것이다
여기서는 람다 함수를 이용해서 각 항목의 'text'를 가져와서 BERT 토크나이저로 토큰화한다

truncation=True : 토큰의 수가 토크나이저의 최대 토큰 수를 초과할 경우 그 부분을 잘라낸다.
batched=True : map()함수가 배치 단위로 데이터를 처리해야 함을 의미
"""



'\nimbd는 load_dataset("imdb")를 통해 로드된 Dataset 객체이다.\nDataset.map(f)는 입력 데이터셋의 각 원소에 함수 f을 적용하여 새로운 데이터셋을 생성해주는 것이다\n여기서는 람다 함수를 이용해서 각 항목의 \'text\'를 가져와서 BERT 토크나이저로 토큰화한다\n\ntruncation=True : 토큰의 수가 토크나이저의 최대 토큰 수를 초과할 경우 그 부분을 잘라낸다.\nbatched=True : map()함수가 배치 단위로 데이터를 처리해야 함을 의미\n'

## 사전 학습된 모델 불러오기

In [4]:
from transformers import AutoModelForSequenceClassification
import torch

In [5]:
# 파이토치 기능을 위한 GPU 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 가장 기본적인 분류를 위한 BERT 모델 로드
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 학습 진행하기

- transformer에서 제공해주는 Trainer 기능 사용 : 하이퍼 파라미터 조건대로 쉽게 모델 학습 진행 가능  

- 가장 긴 길이에 맞춰 padding을 진행하기 위한 collator 선언  

- trainer 내부에서 자동 전처리가 수행

In [6]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # sequence padding
# 배치 내의 시퀀스들의 길이를 동일하게 만들어주는 padding을 자동으로 적용한다

training_args = TrainingArguments(
    output_dir="./checkpoint", # checkpoint 저장 디렉토리
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model = model,  # 위에서 선언했던 BERT 모델
    args=training_args,  # 위에서 지정한 argument 조합
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,  # 자동 전처리 기능
    
)

trainer.train()  # 학습 시작

  0%|          | 0/15625 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.3961, 'learning_rate': 1.936e-05, 'epoch': 0.16}


KeyboardInterrupt: 